In [1]:
from google.colab import files
file = files.upload()

Saving complexity_new.csv to complexity_new.csv
Saving readability_new_data.csv to readability_new_data.csv


In [2]:
!pip install -q \
  transformers\
  datasets\
  scikit-learn \
  accelerate \
  fsspec \
  gcsfs \
  sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import random

### **ROBERTA LARGE MODEL**

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set reproducibility
set_seed(42)



device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


df = pd.read_csv("complexity_new.csv", encoding='ISO-8859-1')

assert 'Sentences' in df.columns, "Missing 'Sentences' column"
df = df.dropna(subset=['Labels'])

label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Labels'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

hf_dataset = Dataset.from_pandas(df[['Sentences', 'Labels']])

model_checkpoint = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def tokenize_function(examples):
    return tokenizer(list(examples["Sentences"]), truncation=True, padding="max_length", max_length=256)


tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("Labels", "label")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)


training_args = TrainingArguments(
    output_dir="./results",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    fp16=True if torch.cuda.is_available() else False,
    seed=42,
    metric_for_best_model="f1",
    save_strategy="steps",
    eval_steps=500,
)


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average="macro"),
        "recall": recall_score(p.label_ids, preds, average="macro"),
        "f1": f1_score(p.label_ids, preds, average="macro"),
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

best_f1 = 0
patience = 2
patience_counter = 0

for epoch in range(training_args.num_train_epochs):
    print(f"Epoch {epoch + 1}/{training_args.num_train_epochs}")
    trainer.train()


    eval_results = trainer.evaluate(test_dataset)
    print("\nEvaluation Results:")
    for metric, value in eval_results.items():
        if isinstance(value, float):
            print(f"{metric.capitalize()}: {value:.4f}")


    if eval_results["eval_f1"] > best_f1:
        best_f1 = eval_results["eval_f1"]
        patience_counter = 0
        trainer.save_model()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered. No improvement in {patience} evaluations.")
            break


Device: cuda
Label Mapping: {'Easy': np.int64(0), 'Hard': np.int64(1), 'Medium': np.int64(2)}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: victorboladejamb17 (victorboladejamb17-university-of-ilorin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Evaluation Results:
Eval_loss: 1.4703
Eval_accuracy: 0.7419
Eval_precision: 0.7278
Eval_recall: 0.6751
Eval_f1: 0.6933
Eval_runtime: 0.3912
Eval_samples_per_second: 79.2420
Eval_steps_per_second: 10.2250
Epoch: 8.0000
Epoch 2/8



Evaluation Results:
Eval_loss: 1.7921
Eval_accuracy: 0.7419
Eval_precision: 0.7116
Eval_recall: 0.7031
Eval_f1: 0.7062
Eval_runtime: 0.3839
Eval_samples_per_second: 80.7590
Eval_steps_per_second: 10.4200
Epoch: 8.0000
Epoch 3/8



Evaluation Results:
Eval_loss: 2.0449
Eval_accuracy: 0.6774
Eval_precision: 0.6614
Eval_recall: 0.6359
Eval_f1: 0.6469
Eval_runtime: 0.3932
Eval_samples_per_second: 78.8440
Eval_steps_per_second: 10.1730
Epoch: 8.0000
Epoch 4/8



Evaluation Results:
Eval_loss: 2.1661
Eval_accuracy: 0.6774
Eval_precision: 0.6614
Eval_recall: 0.6359
Eval_f1: 0.6469
Eval_runtime: 0.4046
Eval_samples_per_second: 76.6180
Eval_steps_per_second: 9.8860
Epoch: 8.0000
Early stopping triggered. No improvement in 2 evaluations.


### **BERT LARGE MODEL**

In [5]:
def set_seed(seed=10):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set reproducibility
set_seed(10)


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


df = pd.read_csv("complexity_new.csv", encoding='ISO-8859-1')
assert 'Sentences' in df.columns, "Missing 'Sentences' column"

df = df.dropna(subset=['Labels'])


label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Labels'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


hf_dataset = Dataset.from_pandas(df[['Sentences', 'Labels']])


model_checkpoint = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(list(examples["Sentences"]), truncation=True, padding="max_length", max_length=256)

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("Labels", "label")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']


num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)


training_args = TrainingArguments(
    output_dir="./results",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    fp16=True if torch.cuda.is_available() else False,
    seed=42,
    save_strategy="steps",
    eval_steps=500,
    metric_for_best_model="f1"
)


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average="macro"),
        "recall": recall_score(p.label_ids, preds, average="macro"),
        "f1": f1_score(p.label_ids, preds, average="macro"),
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


best_f1 = 0
patience = 2
patience_counter = 0

for epoch in range(training_args.num_train_epochs):
    print(f"\nEpoch {epoch + 1}/{training_args.num_train_epochs}")
    trainer.train()
    eval_results = trainer.evaluate(test_dataset)

    print("\nEvaluation Results:")
    for metric, value in eval_results.items():
        if isinstance(value, float):
            print(f"{metric.capitalize()}: {value:.4f}")

    if eval_results["eval_f1"] > best_f1:
        best_f1 = eval_results["eval_f1"]
        patience_counter = 0
        trainer.save_model()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping: No improvement in {patience} evaluations.")
            break


Device: cuda
Label Mapping: {'Easy': np.int64(0), 'Hard': np.int64(1), 'Medium': np.int64(2)}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...


Epoch 1/10



Evaluation Results:
Eval_loss: 0.7413
Eval_accuracy: 0.7097
Eval_precision: 0.7381
Eval_recall: 0.6275
Eval_f1: 0.6594
Eval_runtime: 0.3398
Eval_samples_per_second: 91.2250
Eval_steps_per_second: 5.8850
Epoch: 10.0000

Epoch 2/10



Evaluation Results:
Eval_loss: 1.6354
Eval_accuracy: 0.7097
Eval_precision: 0.7606
Eval_recall: 0.6275
Eval_f1: 0.6655
Eval_runtime: 0.3625
Eval_samples_per_second: 85.5260
Eval_steps_per_second: 5.5180
Epoch: 10.0000

Epoch 3/10



Evaluation Results:
Eval_loss: 2.1439
Eval_accuracy: 0.7097
Eval_precision: 0.7606
Eval_recall: 0.6275
Eval_f1: 0.6655
Eval_runtime: 0.3603
Eval_samples_per_second: 86.0360
Eval_steps_per_second: 5.5510
Epoch: 10.0000

Epoch 4/10



Evaluation Results:
Eval_loss: 2.3497
Eval_accuracy: 0.7097
Eval_precision: 0.7606
Eval_recall: 0.6275
Eval_f1: 0.6655
Eval_runtime: 0.3502
Eval_samples_per_second: 88.5230
Eval_steps_per_second: 5.7110
Epoch: 10.0000

Early stopping: No improvement in 2 evaluations.


## **READABILITY NOTEBOOK**

### **BERT LARGE MODEL**

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


df = pd.read_csv("readability_new_data.csv", encoding='ISO-8859-1')  # use utf-8 if unsure
assert 'Sentences' in df.columns, "Missing 'Sentences' column"


df = df.dropna(subset=['Labels', 'Sentences'])


df['Sentences'] = df['Sentences'].astype(str)


label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Labels'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


hf_dataset = Dataset.from_pandas(df[['Sentences', 'Labels']].reset_index(drop=True))


model_checkpoint = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["Sentences"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("Labels", "label")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']


num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)


training_args = TrainingArguments(
    output_dir="./results",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    fp16=True if torch.cuda.is_available() else False,
    seed=42,
    save_strategy="steps",
    eval_steps=500,
    metric_for_best_model="f1"
)


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average="macro"),
        "recall": recall_score(p.label_ids, preds, average="macro"),
        "f1": f1_score(p.label_ids, preds, average="macro"),
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


best_f1 = 0
patience = 2
patience_counter = 0

for epoch in range(training_args.num_train_epochs):
    print(f"\nEpoch {epoch + 1}/{training_args.num_train_epochs}")
    trainer.train()
    eval_results = trainer.evaluate(test_dataset)

    print("\nEvaluation Results:")
    for metric, value in eval_results.items():
        if isinstance(value, float):
            print(f"{metric.capitalize()}: {value:.4f}")


    if eval_results["eval_f1"] > best_f1:
        best_f1 = eval_results["eval_f1"]
        patience_counter = 0
        trainer.save_model()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping: No improvement in {patience} consecutive epochs.")
            break

Device: cuda
Label Mapping: {'Easy': np.int64(0), 'Hard': np.int64(1), 'Medium': np.int64(2)}


Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/6



Evaluation Results:
Eval_loss: 0.5011
Eval_accuracy: 0.7742
Eval_precision: 0.7842
Eval_recall: 0.7771
Eval_f1: 0.7732
Eval_runtime: 0.3536
Eval_samples_per_second: 87.6630
Eval_steps_per_second: 5.6560
Epoch: 6.0000

Epoch 2/6



Evaluation Results:
Eval_loss: 0.7049
Eval_accuracy: 0.8065
Eval_precision: 0.8268
Eval_recall: 0.8104
Eval_f1: 0.8046
Eval_runtime: 0.3480
Eval_samples_per_second: 89.0700
Eval_steps_per_second: 5.7460
Epoch: 6.0000

Epoch 3/6



Evaluation Results:
Eval_loss: 1.5263
Eval_accuracy: 0.7419
Eval_precision: 0.7833
Eval_recall: 0.7479
Eval_f1: 0.7350
Eval_runtime: 0.3867
Eval_samples_per_second: 80.1750
Eval_steps_per_second: 5.1730
Epoch: 6.0000

Epoch 4/6



Evaluation Results:
Eval_loss: 1.3893
Eval_accuracy: 0.8065
Eval_precision: 0.8268
Eval_recall: 0.8104
Eval_f1: 0.8046
Eval_runtime: 0.3792
Eval_samples_per_second: 81.7560
Eval_steps_per_second: 5.2750
Epoch: 6.0000

Early stopping: No improvement in 2 consecutive epochs.


### **ROBERTA LARGE MODEL**

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


df = pd.read_csv("readability_new_data.csv", encoding='ISO-8859-1')
assert 'Sentences' in df.columns, "Missing 'Sentences' column"


df = df.dropna(subset=['Labels', 'Sentences'])


df['Sentences'] = df['Sentences'].astype(str)


label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Labels'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


hf_dataset = Dataset.from_pandas(df[['Sentences', 'Labels']].reset_index(drop=True))


model_checkpoint = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["Sentences"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("Labels", "label")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']


num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)


training_args = TrainingArguments(
    output_dir="./results",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    fp16=True if torch.cuda.is_available() else False,
    seed=42,
    save_strategy="steps",
    eval_steps=500,
    metric_for_best_model="f1"
)


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average="macro"),
        "recall": recall_score(p.label_ids, preds, average="macro"),
        "f1": f1_score(p.label_ids, preds, average="macro"),
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


best_f1 = 0
patience = 2
patience_counter = 0

for epoch in range(training_args.num_train_epochs):
    print(f"\nEpoch {epoch + 1}/{training_args.num_train_epochs}")
    trainer.train()
    eval_results = trainer.evaluate(test_dataset)

    print("\nEvaluation Results:")
    for metric, value in eval_results.items():
        if isinstance(value, float):
            print(f"{metric.capitalize()}: {value:.4f}")


    if eval_results["eval_f1"] > best_f1:
        best_f1 = eval_results["eval_f1"]
        patience_counter = 0
        trainer.save_model()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping: No improvement in {patience} consecutive epochs.")
            break

Device: cuda
Label Mapping: {'Easy': np.int64(0), 'Hard': np.int64(1), 'Medium': np.int64(2)}


Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/6



Evaluation Results:
Eval_loss: 0.4319
Eval_accuracy: 0.8065
Eval_precision: 0.8109
Eval_recall: 0.8083
Eval_f1: 0.8063
Eval_runtime: 0.3712
Eval_samples_per_second: 83.5200
Eval_steps_per_second: 5.3880
Epoch: 6.0000

Epoch 2/6



Evaluation Results:
Eval_loss: 0.7714
Eval_accuracy: 0.8065
Eval_precision: 0.8120
Eval_recall: 0.8042
Eval_f1: 0.8046
Eval_runtime: 0.3408
Eval_samples_per_second: 90.9640
Eval_steps_per_second: 5.8690
Epoch: 6.0000

Epoch 3/6



Evaluation Results:
Eval_loss: 1.0399
Eval_accuracy: 0.8710
Eval_precision: 0.8761
Eval_recall: 0.8729
Eval_f1: 0.8708
Eval_runtime: 0.3699
Eval_samples_per_second: 83.8090
Eval_steps_per_second: 5.4070
Epoch: 6.0000

Epoch 4/6



Evaluation Results:
Eval_loss: 1.3284
Eval_accuracy: 0.8387
Eval_precision: 0.8403
Eval_recall: 0.8375
Eval_f1: 0.8380
Eval_runtime: 0.3413
Eval_samples_per_second: 90.8340
Eval_steps_per_second: 5.8600
Epoch: 6.0000

Epoch 5/6



Evaluation Results:
Eval_loss: 1.4390
Eval_accuracy: 0.8387
Eval_precision: 0.8403
Eval_recall: 0.8375
Eval_f1: 0.8380
Eval_runtime: 0.3583
Eval_samples_per_second: 86.5240
Eval_steps_per_second: 5.5820
Epoch: 6.0000

Early stopping: No improvement in 2 consecutive epochs.
